## S&P 500 Sector Performance Overview

This notebook loads daily prices for the eleven SPDR sector ETFs (XLC, XLY, XLP, XLE, XLF, XLV, XLI, XLB, XLRE, XLK, XLU), filters the last year of data, and compares their cumulative returns across multiple horizons.

### Data Pipeline
- Download or reuse cached CSVs from `yfinance`, keeping only the `Close` prices per ticker.
- Rename columns so every series shows `Ticker - Sector` in plots and reports.
- Limit the analysis window to the most recent 12 months to highlight current trends.
- Compute simple daily returns with `pct_change()` and roll them into cumulative performance via `(1 + r).cumprod() - 1`.

### Visualization Blocks
1. **Full-year cumulative returns**: compares every sector over the past 12 months.
2. **Last 6 months**: focuses on mid-term rotations and acceleration.
3. **Last 3 months**: captures short-term leadership shifts.

### Key Takeaways
- Identify which sectors lead or lag over each horizon.
- Observe whether leadership is broad-based or concentrated in a few sectors.
- Use the shortened windows to spot emerging trends that may not appear in the full-year chart.

In [1]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [2]:
# portfolio of sector ETFs and their names
ticker_sector_map = {
    "XLC": "Communication Services",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLE": "Energy",
    "XLF": "Financials",
    "XLV": "Health Care",
    "XLI": "Industrials",
    "XLB": "Materials",
    "XLRE": "Real Estate",
    "XLK": "Technology",
    "XLU": "Utilities",
}

tickers = list(ticker_sector_map.keys())
sector_labels = {ticker: f"{ticker} - {name}" for ticker, name in ticker_sector_map.items()}

In [3]:
start_date = "2023-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# download data to csv, filename have tickers joined by underscore and end_date
data.to_csv("_".join(tickers) + "_" + end_date + ".csv")

[*********************100%***********************]  11 of 11 completed


In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("_".join(tickers) + "_" + end_date + ".csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# use descriptive names in charts/legends
df = df.rename(columns=sector_labels)

# Show the result
df.head()

Ticker,XLF - Financials,XLP - Consumer Staples,XLV - Health Care,XLC - Communication Services,XLRE - Real Estate,XLK - Technology,XLE - Energy,XLB - Materials,XLU - Utilities,XLI - Industrials,XLY - Consumer Discretionary
Date,,,,,,,,,,,
2023-01-03,32.703880,68.502548,128.885376,47.062119,33.421204,60.308247,38.125153,36.587341,32.144352,93.992302,62.577797
2023-01-04,33.227982,68.770035,129.227905,47.943142,34.187538,60.464821,38.120644,37.223846,32.436165,94.823166,63.484371
2023-01-05,32.980240,68.078262,127.905365,48.010914,33.186798,59.290550,38.816288,36.634491,31.734001,93.801308,62.894608
2023-01-06,33.713978,69.895287,129.085144,48.795124,34.151478,61.027489,39.552586,37.893364,32.372334,96.370270,64.371460
2023-01-09,33.590096,69.212738,126.925354,48.872578,34.133446,61.736946,39.412560,38.133823,32.591187,95.940521,64.761368


In [5]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [6]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]

In [7]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [8]:
# calculate simple returns with pct_change() for 6m and 3m dataframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for 6m and 3m dataframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1

In [9]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [10]:
# Bar plot of final cumulative returns for each sector
final_returns = cumprod_simple.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns.values,
        y=final_returns.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Returns (Last Year)",
    xaxis_title="Cumulative Return",
    yaxis_title="Sector",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [11]:
# plot last 6 months only
cumprod_simple_last_6_months = cumprod_simple.last('6M')
fig = go.Figure()
for column in cumprod_simple_last_6_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_6_months.index,
            y=cumprod_simple_last_6_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [12]:
# Bar plot of final cumulative returns for each sector (Last 6 Months)
final_returns_6m = cumprod_simple_6m.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_6m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_6m.values,
        y=final_returns_6m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_6m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Returns (Last 6 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Sector",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [13]:
# plot last 3 months only
cumprod_simple_last_3_months = cumprod_simple.last('3M')
fig = go.Figure()
for column in cumprod_simple_last_3_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_3_months.index,
            y=cumprod_simple_last_3_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [14]:
# Bar plot of final cumulative returns for each sector (Last 3 Months)
final_returns_3m = cumprod_simple_3m.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_3m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_3m.values,
        y=final_returns_3m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_3m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Returns (Last 3 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Sector",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [15]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [16]:
# # Bar plot of final cumulative returns for each sector (Last 6 Months - from df_6m)
# final_returns_6m_v2 = cumprod_simple_6m.iloc[-1].sort_values()

# # Create colors based on positive/negative returns
# colors = ['green' if x >= 0 else 'red' for x in final_returns_6m_v2.values]

# fig = go.Figure()
# fig.add_trace(
#     go.Bar(
#         x=final_returns_6m_v2.values,
#         y=final_returns_6m_v2.index,
#         orientation='h',
#         marker_color=colors,
#         text=[f"{x:.1%}" for x in final_returns_6m_v2.values],
#         textposition='outside',
#     )
# )

# fig.update_layout(
#     title="S&P 500 Sectors - Cumulative Returns (Last 6 Months)",
#     xaxis_title="Cumulative Return",
#     yaxis_title="Sector",
#     template="plotly_white",
#     xaxis_tickformat=".0%",
#     height=500,
#     margin=dict(l=200, r=80, t=80, b=60),
# )
# fig.show()

In [17]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [18]:
# # Bar plot of final cumulative returns for each sector (Last 3 Months - from df_3m)
# final_returns_3m_v2 = cumprod_simple_3m.iloc[-1].sort_values()

# # Create colors based on positive/negative returns
# colors = ['green' if x >= 0 else 'red' for x in final_returns_3m_v2.values]

# fig = go.Figure()
# fig.add_trace(
#     go.Bar(
#         x=final_returns_3m_v2.values,
#         y=final_returns_3m_v2.index,
#         orientation='h',
#         marker_color=colors,
#         text=[f"{x:.1%}" for x in final_returns_3m_v2.values],
#         textposition='outside',
#     )
# )

# fig.update_layout(
#     title="S&P 500 Sectors - Cumulative Returns (Last 3 Months)",
#     xaxis_title="Cumulative Return",
#     yaxis_title="Sector",
#     template="plotly_white",
#     xaxis_tickformat=".0%",
#     height=500,
#     margin=dict(l=200, r=80, t=80, b=60),
# )
# fig.show()